## `LinearSelector`, draggable selector that can optionally associated with an ipywidget.

In [ ]:
import fastplotlib as fpl
from fastplotlib.graphics.selectors import Synchronizer

import numpy as np
from ipywidgets import VBox, IntSlider, FloatSlider

plot = fpl.Plot()

# data to plot
xs = np.linspace(0, 100, 1000)
sine = np.sin(xs) * 20

# make sine along x axis
sine_graphic = plot.add_line(np.column_stack([xs, sine]).astype(np.float32))

# make some selectors
selector = sine_graphic.add_linear_selector()
selector2 = sine_graphic.add_linear_selector(20)
selector3 = sine_graphic.add_linear_selector(40)

ss = Synchronizer(selector, selector2, selector3)

def set_color_at_index(ev):
    # changes the color at the index where the slider is
    ix = ev.pick_info["selected_index"]
    g = ev.pick_info["graphic"].parent
    g.colors[ix] = "green"

selector.selection.add_event_handler(set_color_at_index)

# fastplotlib LineSelector can make an ipywidget slider and return it :D 
ipywidget_slider = selector.make_ipywidget_slider()
ipywidget_slider.description = "slider1"

# or you can make your own ipywidget sliders and connect them to the linear selector
ipywidget_slider2 = IntSlider(min=0, max=100, description="slider2")
ipywidget_slider3 = FloatSlider(min=0, max=100, description="slider3")

selector2.add_ipywidget_handler(ipywidget_slider2, step=5)
selector3.add_ipywidget_handler(ipywidget_slider3, step=0.1)

plot.auto_scale()
plot.show(sidecar=True, add_widgets=[ipywidget_slider])

In [ ]:
from pylinalg import vec_transform
from fastplotlib.utils.emulation.events import emulate_pointer_movement

In [ ]:
# get ndc position of selector
ndc = vec_transform(selector.position, plot.camera.camera_matrix)
ndc

In [ ]:
x, y = plot.canvas.get_logical_size()

In [ ]:
screen_matrix = np.array([
    [x / 2, 0, 0, (x - 1) /2],
    [0, y / 2, 0, (y - 1) / 2],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

In [ ]:
# get screen position of selector
screen_pos = vec_transform(ndc, screen_matrix)[:2]
screen_pos

# set an end point for the selector
end_pos = screen_pos.copy()
end_pos[0] += 200  # +100 pixels in x for end position

In [ ]:
emulate_pointer_movement(
    plot.renderer,
    start_position=screen_pos,
    end_position=end_pos,
    button=1,
    modifiers="Shift",
    n_steps=100
)

### Drag linear selectors with the mouse, hold "Shift" to synchronize movement of all the selectors

## Also works for line collections

In [ ]:
sines = [sine] * 10

plot = fpl.Plot()

sine_stack = plot.add_line_stack(sines)

colors = "y", "blue", "red", "green"

selectors = list()
for i, c in enumerate(colors):
    sel = sine_stack.add_linear_selector(i * 100, color=c, name=str(i))
    selectors.append(sel)
    
ss = Synchronizer(*selectors)

plot.show()